In [1]:
import numpy as np
from Baum_Welch_Algorithem import HMM

In [2]:
observations = ('dry','dryish','damp','soggy')
states = ('sunny','cloudy','rainy')

start_probaility = {'sunny':0.6,'cloudy':0.2,'rainy':0.2}

transision_probability = {
    'sunny':{'sunny':0.5,'cloudy':0.375,'rainy':0.125},
    'cloudy':{'sunny':0.25,'cloudy':0.125,'rainy':0.625},
    'rainy':{'sunny':0.25,'cloudy':0.375,'rainy':0.3750}
}

emission_probability = {
    'sunny':{'dry':0.6,'dryish':0.2,'damp':0.15,'soggy':0.05},
    'cloudy':{'dry':0.25,'dryish':0.25,'damp':0.25,'soggy':0.25},
    'rainy':{'dry':0.05,'dryish':0.1,'damp':0.35,'soggy':0.50}
}

In [3]:
def generate_index_map(labels):
    index_label = {}
    label_index = {}

    i = 0
    for l in labels:
        index_label[i] = l
        label_index[l] = i

        i += 1
    return label_index,index_label

states_label_index,states_index_label = generate_index_map(states)
observasions_label_indec,observations_index_label = generate_index_map(observations)

def convert_observations_to_index (observations,label_index):
    list = []
    for o in observations:
        list.append(label_index[o])
    return list

def convert_map_to_matrix (probability_map,label_index1,label_index2):
    m = np.empty((len(label_index1),len(label_index2)),dtype = float)
    for line in probability_map:
        for col in probability_map[line]:
            m[label_index1[line]][label_index2[col]] = probability_map[line][col]
    return m

def convert_map_to_vector (map,label_index):
    v = np.empty(len(map),dtype = float)
    for e in map:
              v[label_index[e]] = map[e]
    return v


A = convert_map_to_matrix(transision_probability,states_label_index,states_label_index)
print(A)
B = convert_map_to_matrix(emission_probability,states_label_index,observasions_label_indec)
print(B)
observations_index = convert_observations_to_index(observations,observasions_label_indec)
print(observations_index)
pi = convert_map_to_vector(start_probaility,states_label_index)
print(pi)

h = HMM(A,B,pi)

obs_seq = ('dry','damp','soggy')
obs_seq_indes = convert_observations_to_index(obs_seq,observasions_label_indec)
print(obs_seq_indes)

F = h._forward(obs_seq_indes)
print("forward: P(O|lambda) = %f" %sum(F[:,-1]))
X = h._backward(obs_seq_indes)
print("backward: P(O|lambda) = %f" %sum(X[:,0]*pi*B[:,0]))


[[0.5   0.375 0.125]
 [0.25  0.125 0.625]
 [0.25  0.375 0.375]]
[[0.6  0.2  0.15 0.05]
 [0.25 0.25 0.25 0.25]
 [0.05 0.1  0.35 0.5 ]]
[0, 1, 2, 3]
[0.6 0.2 0.2]
[0, 2, 3]
forward: P(O|lambda) = 0.026441
backward: P(O|lambda) = 0.026441


In [4]:
#计算P(i|o)
p,ss = h.stat_path(obs_seq_indes)
path = []
for s in ss:
    path.append(states_index_label[s])

print("最有可能的隐藏序列",path)
print("viterbi:P(I|O) = %f"%p)

最有可能的隐藏序列 ['sunny', 'cloudy', 'rainy']
viterbi:P(I|O) = 0.010547


In [5]:
observation_data,states_data = h.simulate(100)
guess = HMM(np.array([[0.33,0.33,0.34],
                   [0.33,0.33,0.34],
                   [0.33,0.33,0.34]]),
            np.array([[0.25,0.25,0.25,0.25],
                   [0.25,0.25,0.25,0.25],
                   [0.25,0.25,0.25,0.25]]),
            np.array([0.33,0.33,0.34])
               )
guess.baum_welch_train(observation_data)

#预测问题
states_out = guess.stat_path(observation_data)[1]
p = 0.0
for s in states_data:
    if next(states_out) == s:p += 1

print(p/len(states_data))

0.45


In [6]:
A = np.array( [[0.5, 0.375, 0.125],
              [0.25, 0.125, 0.625],
              [0.25, 0.375, 0.375]])
B = np.array( [[0.6, 0.2, 0.15, 0.05],
              [0.25, 0.25, 0.25, 0.25],
              [0.05, 0.1, 0.35, 0.5 ]])
pi =np.array( [0.6, 0.2, 0.2])
h = HMM(A,B,pi)

observation_data,states_data = h.simulate(100)

guess = HMM(np.array([[0.33,0.33,0.34],
                   [0.33,0.33,0.34],
                   [0.33,0.33,0.34]]),
            np.array([[0.25,0.25,0.25,0.25],
                   [0.25,0.25,0.25,0.25],
                   [0.25,0.25,0.25,0.25]]),
            np.array([0.33,0.33,0.34])
               )
guess.baum_welch_train(observation_data)

#预测问题
states_out = guess.stat_path(observation_data)[1]
p = 0.0
for s in states_data:
    if next(states_out) == s:p += 1

print(p/len(states_data))

0.15


In [7]:
A = np.array( [[0.5, 0.375, 0.125],
              [0.25, 0.125, 0.625],
              [0.25, 0.375, 0.375]])
print(A[:,-1])

[0.125 0.625 0.375]
